In [1]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoTokenizer

c:\Users\white\anaconda3\envs\style\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 데이터 로드

In [2]:
name = 'data/inmoon.csv'

df = pd.read_csv(name)

### 토크나이징 & 임베딩 모델 로드

In [3]:
# Llama3 모델의 토크나이저를 불러옵니다.
model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding='max_length', max_length=100, truncation=True)

# 임베딩 레이어를 정의합니다.
embedding_layer = nn.Embedding.from_pretrained(torch.empty((128256, 4096)))  # 크기는 예시입니다.

# 임베딩 레이어를 추출한 후 이를 별도의 nn.Module로 래핑합니다.
class EmbeddingsOnlyModel(nn.Module):
    def __init__(self, embedding_layer):
        super(EmbeddingsOnlyModel, self).__init__()
        self.embeddings = embedding_layer

    def forward(self, input_ids):
        return self.embeddings(input_ids)

# 래핑된 모델 인스턴스를 생성합니다.
embeddings_only_model = EmbeddingsOnlyModel(embedding_layer)

# 저장된 모델 상태를 불러옵니다.
model_path = "embeddings_only_model.pth"
embeddings_only_model.load_state_dict(torch.load(model_path))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<All keys matched successfully>

### 토큰화

In [4]:
def tokenize_text(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=100, return_tensors='pt')['input_ids'].squeeze().tolist()


df['llama_token'] = df['src_sentence'].apply(tokenize_text)

In [5]:
len(df["llama_token"][1])

100

In [6]:
# 모델을 병렬화합니다.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embeddings_only_model = nn.DataParallel(embeddings_only_model)
embeddings_only_model.to(device)

# 임베딩 함수 정의
def embed_text(token_ids_batch):
    input_ids = torch.tensor(token_ids_batch, dtype=torch.long).to(device)
    with torch.no_grad():
        embeddings = embeddings_only_model(input_ids).cpu().numpy()
    return embeddings

# 배치 단위로 토큰화된 텍스트를 임베딩합니다.
batch_size = 1024  # 배치 크기 설정
embeddings_list = []
for i in range(0, len(df), batch_size):
    print(f"{i} / {len(df)}")
    batch_token_ids = df["llama_token"].iloc[i:i+batch_size].tolist()
    batch_embeddings = embed_text(batch_token_ids)
    embeddings_list.extend(batch_embeddings)

# 임베딩된 결과를 embedding 열에 추가합니다.
df["bllossom_embedding"] = embeddings_list


0 / 271721
1024 / 271721
2048 / 271721
3072 / 271721
4096 / 271721
5120 / 271721
6144 / 271721
7168 / 271721
8192 / 271721
9216 / 271721
10240 / 271721
11264 / 271721
12288 / 271721
13312 / 271721
14336 / 271721
15360 / 271721
16384 / 271721
17408 / 271721
18432 / 271721
19456 / 271721
20480 / 271721
21504 / 271721
22528 / 271721
23552 / 271721
24576 / 271721
25600 / 271721
26624 / 271721
27648 / 271721
28672 / 271721
29696 / 271721
30720 / 271721
31744 / 271721
32768 / 271721
33792 / 271721
34816 / 271721
35840 / 271721
36864 / 271721
37888 / 271721
38912 / 271721
39936 / 271721
40960 / 271721
41984 / 271721
43008 / 271721
44032 / 271721
45056 / 271721
46080 / 271721
47104 / 271721
48128 / 271721
49152 / 271721
50176 / 271721
51200 / 271721
52224 / 271721
53248 / 271721
54272 / 271721
55296 / 271721
56320 / 271721
57344 / 271721
58368 / 271721
59392 / 271721
60416 / 271721
61440 / 271721
62464 / 271721
63488 / 271721
64512 / 271721
65536 / 271721
66560 / 271721
67584 / 271721
68608 / 

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1677721600 bytes.